# Project : OMR scanner and test grader    

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
 pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [4]:
#importing all the necessary liabrary 
import numpy as np   # For numerical operations.
import cv2           #OpenCV library for image processing
import imutils       #Utility functions for image processing
from imutils import contours #For contour sorting

In [5]:
# order_points(pts): Orders points in a consistent manner (top-left, top-right, bottom-right, bottom-left).
def order_points(pts):
    """
    Order points in the order: top-left, top-right, bottom-right, bottom-left.
    Args:
    pts: Array of four points (x, y) from the image.

    Returns:
    rect: Array of points ordered as top-left, top-right, bottom-right, bottom-left.
    """
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect


In [8]:
# four_point_transform(image, pts): Applies a perspective transform to get a top-down view of the image.
def four_point_transform(image, pts):
    """
    Apply a perspective transform to obtain a top-down view of the image.
    Args:
    image: Input image.
    pts: Array of four points (x, y) representing the corners of the object.

    Returns:
    warped: The warped (transformed) image.
    """
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
# Calculate the width of the new image
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

# Calculate the height of the new image
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

# Set of destination points for "bird's eye view"
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

# Compute the perspective transform matrix and apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return warped

In [9]:
# Load and preprocess the image
image_path = r"C:\Users\Shubh\Downloads\omr.png"
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"The image file {image_path} could not be loaded. Please check the file path.")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to the grayscale image
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Perform edge detection
edged = cv2.Canny(blurred, 75, 200)

# Find contours in the edge map
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

# Ensure at least one contour was found
if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # Loop over the sorted contours
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        # If the contour has four points, assume it is the document
        if len(approx) == 4:
            docCnt = approx
            break

if docCnt is None:
    raise ValueError("No document contour found in the image.")

# Apply the four-point transform to obtain a top-down view of the paper
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

# Apply thresholding to the warped grayscale image
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# Find contours in the thresholded image
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []

# Loop over the contours
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    # Filter out contours that are not circular
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

In [10]:
# Sort the question contours top-to-bottom
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
correct = 0
Answer_key = {0: 1, 1: 4, 2: 0, 3: 2, 4: 0}

# Loop over each question in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = None

    # Loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # Create a mask that reveals only the current bubble for counting
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)

        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    color = (0, 0, 255)
    k = Answer_key[q]

    # Check if the bubbled answer is correct
    if k == bubbled[1]:
        correct += 1

print(f"Correct Answers: {correct}")

Correct Answers: 4


In [ ]:
dst1 = np.array([[0, 0],[maxWidth - 1, 0],[maxWidth - 1, maxHeight - 1],[0, maxHeight - 1]], dtype="float32")